In [1]:
# import io
import docx
import os
# import pytesseract
from tabulate import tabulate
from PIL import Image

In [2]:
import sys

In [3]:
sys.path.insert(0, "./../")

In [4]:
# doc_path = "./../documents/尼得科CT变频器推广使用说明.docx"
doc_path = "./../documents/健鼎机组蒸汽回水pH控制器设计更改事宜.docx"

In [5]:
doc = docx.Document(doc_path)

In [93]:
def extract_tables(doc):
    tables = []
    for table in doc.tables:
        table_data = []
        for row in table.rows:
            row_data = []
            for cell in row.cells:
                cell_text = ''
                for paragraph in cell.paragraphs:
                    cell_text += paragraph.text
                row_data.append(cell_text)
            table_data.append(row_data)

        tables.append(tabulate(table_data, tablefmt="plain"))
    return tables

In [94]:
tables = extract_tables(doc)

In [96]:
def extract_texts(doc):
    texts = []
    for p in doc.paragraphs:
        texts.append(p.text)
    return texts

In [97]:
elements = []
table_indices = []
for index, element in enumerate(doc.element.body):
    if element.tag.endswith("tbl"):
        table_indices.append(index)
    elements.append(element.text)

In [105]:
element.index()

TypeError: index() takes at least 1 positional argument (0 given)

In [98]:
for index, table in enumerate(tables):
    table_index = table_indices[index]
    elements[table_index] = table

In [50]:
elements = [element for element in elements if isinstance(element, str)]

In [51]:
def consolidate_elements(elements, sentence_len=120, overlapping_size=2):

    final_sentence_list = []
    accumulate_len = 0
    windows_sentences = []

    for sentence in elements:

        word_len = len(sentence)
        if accumulate_len+word_len <= sentence_len:
            windows_sentences.append(sentence)
            accumulate_len += word_len
        else:
            windows_sentence = "\n".join(windows_sentences)
            final_sentence_list.append(windows_sentence)
            windows_sentences = windows_sentences[-overlapping_size:].copy()+[
                sentence]
            accumulate_len = word_len

    if len(windows_sentences) > 0:
        windows_sentence = "\n".join(windows_sentences)
        final_sentence_list.append(windows_sentence)

    return final_sentence_list

In [52]:
chunks = consolidate_elements(elements, sentence_len=256, overlapping_size=2)

In [53]:
# print(chunks[2])

In [54]:
from app.routers.documents.embedding import SentencesToEmbeddings

In [55]:
sentences_to_vectors = SentencesToEmbeddings()

In [56]:
# sentences_to_vectors(chunks)

In [57]:
import chromadb

In [58]:
db_path = "/Users/nelsonlin/Desktop/workspaces/llm-local-chat-api/app/data"

In [59]:
chroma_client = chromadb.PersistentClient(path=db_path)

In [60]:
document_collection = chroma_client.get_or_create_collection(name="documents")

In [61]:
file_path = os.path.abspath(doc_path)
file_name = os.path.basename(doc_path)

In [62]:
document_collection.add(
    documents=[file_name],
    embeddings=[[0]],
    metadatas={"file_name": file_name, "file_path": file_path},
    ids=[file_name]
)

In [63]:
document_collection.get()

{'ids': ['健鼎机组蒸汽回水pH控制器设计更改事宜.docx', '尼得科CT变频器推广使用说明.docx'],
 'embeddings': None,
 'metadatas': [{'file_name': '健鼎机组蒸汽回水pH控制器设计更改事宜.docx',
   'file_path': '/Users/nelsonlin/Desktop/workspaces/llm-local-chat-api/documents/健鼎机组蒸汽回水pH控制器设计更改事宜.docx'},
  {'file_name': '尼得科CT变频器推广使用说明.docx',
   'file_path': '/Users/nelsonlin/Desktop/workspaces/llm-local-chat-api/documents/尼得科CT变频器推广使用说明.docx'}],
 'documents': ['健鼎机组蒸汽回水pH控制器设计更改事宜.docx', '尼得科CT变频器推广使用说明.docx'],
 'uris': None,
 'data': None}

In [64]:
document_collection.get(where={"file_name": file_name})

{'ids': ['健鼎机组蒸汽回水pH控制器设计更改事宜.docx'],
 'embeddings': None,
 'metadatas': [{'file_name': '健鼎机组蒸汽回水pH控制器设计更改事宜.docx',
   'file_path': '/Users/nelsonlin/Desktop/workspaces/llm-local-chat-api/documents/健鼎机组蒸汽回水pH控制器设计更改事宜.docx'}],
 'documents': ['健鼎机组蒸汽回水pH控制器设计更改事宜.docx'],
 'uris': None,
 'data': None}

In [65]:
# document_collection.delete(where={"file_name": file_name})

In [66]:
embedding_collection = chroma_client.get_or_create_collection(
    name="documents_embedding")

In [67]:
# document_collection.delete(where={"file_name": file_name})

In [68]:
document_collection.get()

{'ids': ['健鼎机组蒸汽回水pH控制器设计更改事宜.docx', '尼得科CT变频器推广使用说明.docx'],
 'embeddings': None,
 'metadatas': [{'file_name': '健鼎机组蒸汽回水pH控制器设计更改事宜.docx',
   'file_path': '/Users/nelsonlin/Desktop/workspaces/llm-local-chat-api/documents/健鼎机组蒸汽回水pH控制器设计更改事宜.docx'},
  {'file_name': '尼得科CT变频器推广使用说明.docx',
   'file_path': '/Users/nelsonlin/Desktop/workspaces/llm-local-chat-api/documents/尼得科CT变频器推广使用说明.docx'}],
 'documents': ['健鼎机组蒸汽回水pH控制器设计更改事宜.docx', '尼得科CT变频器推广使用说明.docx'],
 'uris': None,
 'data': None}

In [69]:
embedding_collection.get()

{'ids': ['健鼎机组蒸汽回水pH控制器设计更改事宜.docx_0',
  '健鼎机组蒸汽回水pH控制器设计更改事宜.docx_1',
  '健鼎机组蒸汽回水pH控制器设计更改事宜.docx_2',
  '健鼎机组蒸汽回水pH控制器设计更改事宜.docx_3'],
 'embeddings': None,
 'metadatas': [{'file_name': '健鼎机组蒸汽回水pH控制器设计更改事宜.docx',
   'file_path': '/Users/nelsonlin/Desktop/workspaces/llm-local-chat-api/documents/健鼎机组蒸汽回水pH控制器设计更改事宜.docx'},
  {'file_name': '健鼎机组蒸汽回水pH控制器设计更改事宜.docx',
   'file_path': '/Users/nelsonlin/Desktop/workspaces/llm-local-chat-api/documents/健鼎机组蒸汽回水pH控制器设计更改事宜.docx'},
  {'file_name': '健鼎机组蒸汽回水pH控制器设计更改事宜.docx',
   'file_path': '/Users/nelsonlin/Desktop/workspaces/llm-local-chat-api/documents/健鼎机组蒸汽回水pH控制器设计更改事宜.docx'},
  {'file_name': '健鼎机组蒸汽回水pH控制器设计更改事宜.docx',
   'file_path': '/Users/nelsonlin/Desktop/workspaces/llm-local-chat-api/documents/健鼎机组蒸汽回水pH控制器设计更改事宜.docx'}],
 'documents': ['宇宙集团\n联络单\nQR.803.02-04(VER:B)                                                         编号：LL-D2401231',
  '联络单\nQR.803.02-04(VER:B)                                                         编号：LL-D24

In [70]:
metadatas = [{'file_path': file_path, 'file_name': file_name}] * len(chunks)

In [71]:
embeddings = sentences_to_vectors(chunks)

In [72]:
ids = [file_name+f"_{i}" for i in range(len(chunks))]

In [73]:
len(embeddings)

4

In [74]:
embedding_collection.add(
    documents=chunks,
    embeddings=embeddings,
    metadatas=metadatas,
    ids=ids
)

Insert of existing embedding ID: 健鼎机组蒸汽回水pH控制器设计更改事宜.docx_0
Insert of existing embedding ID: 健鼎机组蒸汽回水pH控制器设计更改事宜.docx_1
Insert of existing embedding ID: 健鼎机组蒸汽回水pH控制器设计更改事宜.docx_2
Insert of existing embedding ID: 健鼎机组蒸汽回水pH控制器设计更改事宜.docx_3
Add of existing embedding ID: 健鼎机组蒸汽回水pH控制器设计更改事宜.docx_0
Add of existing embedding ID: 健鼎机组蒸汽回水pH控制器设计更改事宜.docx_1
Add of existing embedding ID: 健鼎机组蒸汽回水pH控制器设计更改事宜.docx_2
Add of existing embedding ID: 健鼎机组蒸汽回水pH控制器设计更改事宜.docx_3


In [76]:
print(chunks[3])

PH控制器参数设置：
1）高点报警设置：继电器1(Relay 1)把设定点(SP)设置为8.00pH，调整迟滞值(Hys)设置为 0.10pH；
2）低点报警设置：继电器2(Relay 2)把设定点(SP)设置为5.00pH，调整迟滞值(Hys)设置为0.10pH。根据操作手册进入操作页面设置如下图：




请服务部根据第1、2点的要求进行更改，共有67台机需要更改，清单如下表格：

如有不明之处，请及时联系！


In [77]:
query_text = "PH控制器参数设置"

In [78]:
query_embedding = sentences_to_vectors([query_text])[0]

In [86]:
embedding_query_results = embedding_collection.query(query_embeddings=query_embedding,
                                                     where={
                                                         "file_name": file_name})

Number of requested results 10 is greater than number of elements in index 4, updating n_results = 4


In [113]:
print(
    "\n".join(embedding_query_results['documents'][0]))

PH控制器参数设置：
1）高点报警设置：继电器1(Relay 1)把设定点(SP)设置为8.00pH，调整迟滞值(Hys)设置为 0.10pH；
2）低点报警设置：继电器2(Relay 2)把设定点(SP)设置为5.00pH，调整迟滞值(Hys)设置为0.10pH。根据操作手册进入操作页面设置如下图：




请服务部根据第1、2点的要求进行更改，共有67台机需要更改，清单如下表格：

如有不明之处，请及时联系！
QR.803.02-04(VER:B)                                                         编号：LL-D2401231
主题  主题        健鼎机组蒸汽回水pH控制器设计更改事宜  健鼎机组蒸汽回水pH控制器设计更改事宜  健鼎机组蒸汽回水pH控制器设计更改事宜  批准
版次  发出日期    编制者                                类型选择                              制品处理                              制品处理      厂外旧设计损坏后维修方案
R0    2024/01/23  袁杰                                  无                                    无                                    按联络单执行
大家好：
服务部反馈健鼎机组蒸汽回水管路上安装的pH控制器设计的报警功能不完善，因实际使用过程中会检测酸性药水也会检测碱性药水，但是电路图只设计了一个高点报警输出，会导致检测酸性药水时没有报警的情况，为此请服务部按以下要求更改：
pH控制器按下图更改,增加一路低点报警。
      
PH控制器参数设置：
1）高点报警设置：继电器1(Relay 1)把设定点(SP)设置为8.00pH，调整迟滞值(Hys)设置为 0.10pH；
联络单
QR.803.02-04(VER:B)                                                         编号：LL-D2401231
主题  主题        健鼎机组蒸汽回水pH控制器设计更改事宜  健

In [83]:
# embedding_query_results = embedding_collection.query(query_embeddings=query_embedding,
#                                                      where={
#                                                          "file_name": "sdf"})a

Number of requested results 10 is greater than number of elements in index 4, updating n_results = 4


In [85]:
embedding_query_results['documents']

[[]]

In [65]:
len(embedding_query_results['documents'])

1

In [66]:
embedding_query_results['documents'][0]

[]